# Tenencia del producto préstamo en el DataSet de Banco Checo  

Vamos a intentar extraer del datset generado que variables son las más relevantes para que una cuenta (account) tenga un péstamo (loan) y ver si de esta forma podemos generar un customer journey para conseguir que un cliente contrate un préstamo

In [ ]:
import pandas as pd
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import numpy.core.multiarray

In [ ]:
#Para garantizar la replicabilidad del análisis
np.random.seed()

In [ ]:
np.random.seed?

In [ ]:
# Cargamos los datos que hemos procesado en R
df_original= pd.read_csv("C://Master Data Science/Master en Data Science/TFM/Transacciones de Banco Checo/DFTenenciaProductos2.csv")

In [ ]:
#Comenzamos a revisar que el DF se haya importado correctamente
df_original.shape

In [ ]:
df_original.head()

In [ ]:
# Vamos a analizar si hay missings al cargar los datos a Python
df_original.columns[df_original.isnull().sum()!=0]
#Vemos que las variables con missings provienen de variables que ya tenían esos missings en el DataFrame generado con R,
#ya que el disponent (autorizado), los préstamos y las tarjetas no son productos que tengan asociados todas las cuentas  

In [ ]:
#Vemos como se han importado las variables del DataFrame de R al DataFrame que vamos a utilizar en Python 
print(df_original.iloc[:,0:32].dtypes)
print(df_original.iloc[:,31:61].dtypes)
print(df_original.iloc[:,60:70].dtypes)
#Observamos que las variables de factor y de fecha han modificado su tipo de variable, 
#por lo que tendremos que trabajar con ellas

In [ ]:
#Transformamos a formato fecha las variables que originalmente eran fecha en R
df_original[["Date_Account","birth_owner", "birth_disponent", "Date_Loan", "owner_card_date"]]=df_original[["Date_Account","birth_owner", "birth_disponent", "Date_Loan", "owner_card_date"]].apply(pd.to_datetime)
from datetime import datetime
df_original['Date_Account']=df_original['Date_Account'].apply(datetime.toordinal)
df_original['birth_owner']=df_original['birth_owner'].apply(datetime.toordinal)
df_original['birth_disponent']=df_original['birth_disponent'].apply(datetime.toordinal)
df_original['Date_Loan']=df_original['Date_Loan'].apply(datetime.toordinal)
df_original['owner_card_date']=df_original['owner_card_date'].apply(datetime.toordinal)

In [ ]:
#De todas las variables de las que disponemos, vamos a seleccionar las que vamos a utilizar en este ejercicio:

#La  variable que vamos a predecir va a ser "account_loan_bin" y por tanto la denominaremos "y"

#Para este ejercicio no consideramos las variables que hemos obtenido en el mismo fichero que la variable a predecir (loan.csv):
#loan_id, Date_Loan, Amount_Loan, Duration_Loan, Payments_Loan, status, Status_Loan.

#Otras variables que no consideramos: account_id, client_id_owner, client_id_disponent, district ID y unnamed:0 porque 
#son ID's descriptivas sin información para utilizar

#La variable disponent_card_type no aporta información

#Otras variables que no consideramos:district_name, region (nombre). Utilizamos el resto de la información de variables 
#del fichero district.csv

#Haciendo referencia a district.csv , la información de crimes_95, crimes_96 y entrepreneurs la vamos a utilizar como ratio,
#ya que como veremos a continuación, de esta forma obtendremos menos impacto por un distrito con mucha población (Praga) 



In [ ]:
#Las variables crimes_95, crimes_96 y entrepreneurs vamos a utilizarlas en formato ratio, tal y como calculamos en R

In [ ]:
plt.hist(df_original['crimes_95'])

In [ ]:
plt.hist(df_original['crimes_95_ratio'])

In [ ]:
plt.hist(df_original['crimes_96'])

In [ ]:
plt.hist(df_original['crimes_96_ratio'])

In [ ]:
plt.hist(df_original['entrepreneurs'])

In [ ]:
plt.hist(df_original['entrepreneurs_ratio'])

In [ ]:
#Creamos un DataFrame con las variables que vamos a considerar numéricas
df_num=df_original[['Date_Account','birth_owner', 'birth_disponent','owner_card_date','Ord_Insurance', 'Ord_Insurance_amount',
                    'Ord_Household_Payment','Ord_Household_Payment_amount', 'Ord_Loan_Payment', 'Ord_Leasing',
                    'Ord_Empty', 'Ord_Empty_amount', 'num_inhabitants', 'municip < 499', 'municip 500-1999',
                    'municip 2000-9999', 'municip > 10000', 'num_cities', 'avg_salary',  
                    'Num_Type_Credit', 'Num_Type_VYBER', 'Num_Type_Withdrawal', 'Num_Op_Null', 'Num_Op_Remittances',
                    'Num_Op_Collection','Num_Op_CashCredit', 'Num_Op_WithdrawalCash','Num_Op_WithdrawalCreditCard',
                    'Num_Sym_Null', 'Num_Sym_Null2','Num_Sym_Pension', 'Num_Sym_Insurance', 'Num_Sym_NegBal',
                    'Num_Sym_Household', 'Num_Sym_Statement', 'Num_Sym_IntDep', 'Num_Sym_LoanPayment', 
                    'Balance_in_negative','Ord_Loan_Payment_amount', 'Ord_Leasing_amount','ratio_urban_inhabitants',
                    'unemployment_rate_95','unemployment_rate_96', 'crimes_95_ratio', 'crimes_96_ratio', 'entrepreneurs_ratio' ]] 

In [ ]:
#Creamos un DataFrame con las variables que vamos a considerar categóricas
df_cat=df_original[['account_disponent_bin','frequency', 'sex_owner', 'owner_card_type',
       'sex_disponent']]
#Vemos que tipos tienen las variables que queremos que sean categóricas
df_cat.dtypes

In [ ]:
#Ponemos las variables "owner_card_type" y "account_disponent_bin" como string para poder obtener dummies
df_cat["owner_card_type"]=df_cat["owner_card_type"].astype(str)
df_cat["account_disponent_bin"]=df_cat["account_disponent_bin"].astype(str)

In [ ]:
df_cat_dumm=pd.get_dummies(df_cat)

In [ ]:
df_cat_dumm.dtypes
#Al pasar a dummies las variables, hemos incrementado en 8 el número total de variables

In [ ]:
df = pd.concat([df_num, df_cat_dumm], axis = 1)
df.head()

# Vamos a generar un primer modelo benchmark

In [ ]:
#Antes de probar con modelos vamos a ver si reduciendo dimensionalidad conseguimos una primera intuición

In [ ]:
X = df
y = df_original["account_loan_bin"] 

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2).fit_transform(X)
plt.figure(dpi=120)
plt.scatter(pca[y.values==0,0], pca[y.values==0,1], alpha=0.5, label='NO', s=2, color='navy')
plt.scatter(pca[y.values==1,0], pca[y.values==1,1], alpha=0.5, label='YES', s=2, color='darkorange')
plt.legend()
plt.title('Producto préstamo')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
#Del gráfico anterior no consigo sacar nada en claro
pca2 = PCA(n_components=2)
pca2.fit(X)
print(pca2.components_)
print(pca2.explained_variance_ratio_)
#De los componentes de momento tampoco sacamos ninguna conclusión. Las variables están en escalas muy distintas y por eso 
#el PCA genera resultados tan "positivos" en cuanto a explicabilidad de las 2 primeras componentes principales

In [ ]:
#Pasamos a hacer modelos sencillos
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
#Generamos conjuntos de train y de test. Para el test usamos el 20% de las observaciones
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
#Probamos una Regresión logística
clf_LR=LogisticRegression(random_state=0)
clf_LR.fit(X_train,y_train)

In [ ]:
def eval_modelo (clf,X_train,y_train, X_test,y_test):
    print("Datos de train:")
    print("El accuracy es",accuracy_score(y_train,clf.predict(X_train))*100,"%")
    print("La precision es",precision_score(y_train,clf.predict(X_train))*100, "%")
    print("El recall es",recall_score(y_train,clf.predict(X_train))*100,"%")
    tn, fp, fn, tp=confusion_matrix(y_train,clf.predict(X_train)).ravel()
    print("tn:",tn," fp:",fp," fn:",fn," tp:",tp)
    print("Datos de test:")
    print("El accuracy es",accuracy_score(y_test,clf.predict(X_test))*100,"%")
    print("La precision es",precision_score(y_test,clf.predict(X_test))*100, "%")
    print("El recall es",recall_score(y_test,clf.predict(X_test))*100,"%")
    tn_t, fp_t, fn_t, tp_t=confusion_matrix(y_test,clf.predict(X_test)).ravel()
    print("tn:",tn_t," fp:",fp_t," fn:",fn_t," tp:",tp_t)      

In [ ]:
eval_modelo(clf_LR,X_train,y_train, X_test,y_test)

In [ ]:
#Probamos con un árbol de decisión
clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_tree.fit(X_train,y_train)

In [ ]:
eval_modelo(clf_tree,X_train,y_train, X_test,y_test)

In [ ]:
#Los resultados de estos modelos tan sencillos están siendo extraordinarios. Vamos a investigar las razones.
#Además como las clases están desbalanceadas vamos a ir comparando el efecto de incluir oversampling o no. 
#Vamos a profundizar con Decission Trees en este primer momento ya que tiene menores requerimientos teóricos para las features 

In [ ]:
#Las clases a predecir (si la cuenta tiene prestamo=1 ó no tiene =0) están desbalanceadas
y2=pd.DataFrame(y)
sns.countplot(x="account_loan_bin",data=y2, palette='hls')
plt.show

In [ ]:
y2['account_loan_bin'].value_counts()

In [ ]:
len(y2['account_loan_bin'])

In [ ]:
#Veamos el porcentaje que representa cada clase:
print("Las cuentas CON préstamo son el", "%.2f" % (y2['account_loan_bin'].value_counts()[0]/len(y2['account_loan_bin'])*100) ,"%")
print("Las cuentas SIN préstamo son el", "%.2f" % (y2['account_loan_bin'].value_counts()[1]/len(y2['account_loan_bin'])*100) ,"%")

In [ ]:
#Al estar las clases desbalanceadas hay que ir con cuidado porque un modelo que prediga siempre NO tendría un accuracy
#de casi el 85%

In [ ]:
#Vamos a querer dibujar Decision Trees
from io import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus

In [ ]:
#Función para dibujar un árbol
def dibu_arb(tree):
    dot_data = StringIO()
    export_graphviz(tree, out_file=dot_data,filled=True, rounded=True,
                special_characters=True)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    return(Image(graph.create_png()))
    

In [ ]:
dibu_arb(clf_tree)

In [ ]:
X_train.columns[36]
#Parece que la variable "Num_Sym_LoanPayment" contiene toda la información "account_loan_bin", 
#aunque se han extraido de ficheros distintos

In [ ]:
X.columns[clf_tree.feature_importances_>0.10] #Vamos a ver las variables más importantes para el modelo

In [ ]:
clf_tree.feature_importances_

In [ ]:
#Link interesante para ver como se realiza el cálculo de "feature_importances"_
#https://medium.com/@srnghn/the-mathematics-of-decision-trees-random-forest-and-feature-importance-in-scikit-learn-and-spark-f2861df67e3

In [ ]:
# conda install -c glemaitre imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTENC

In [ ]:
#Para ver las columnas que vamos a denominar como categóricas cuando apliquemos SMOTE
print(X_train.columns)

In [ ]:
smo=SMOTENC(categorical_features=range(46,59),random_state=0)#Las variables categóricas van a ser de la 46 a la 59
os_X,os_y=smo.fit_sample(X_train, y_train)
columns = X_train.columns
os_X = pd.DataFrame(data=os_X,columns=columns)
os_y= pd.DataFrame(data=os_y,columns=['account_loan_bin'])

In [ ]:
#Chequeamos que SMOTENC funciona como esperábamos

print("length of oversampled data is ",len(os_X))
print("Number of loans=0 in oversampled data",len(os_y[os_y['account_loan_bin']==0]))
print("Number of loans=1",len(os_y[os_y['account_loan_bin']==1]))
print("Proportion of loans=0 is ",len(os_y[os_y['account_loan_bin']==0])/len(os_X))
print("Proportion of loans=1 is ",len(os_y[os_y['account_loan_bin']==1])/len(os_X))

os_bin=os_X[['account_disponent_bin_0','account_disponent_bin_1',
       'frequency_After_trans', 'frequency_Monthly', 'frequency_Weekly',
       'sex_owner_F', 'sex_owner_M', 'owner_card_type_0', 'owner_card_type_1',
       'owner_card_type_2', 'owner_card_type_3', 'sex_disponent_F',
       'sex_disponent_M']]
print("unique de variables categóricas",unique(os_bin))
print("unique de variable y",unique(os_y))

In [ ]:
os_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_tree_os=os_clf_tree.fit(os_X,os_y)

In [ ]:
eval_modelo(clf_tree_os,os_X,os_y, X_test, y_test)
#Obtenemos los mismos resultados con y sin SMOTE. Lógicamente porque estamos suponiendo que la variable 
#'Num_Sym_LoanPayment' contiene la información de si la cuenta tiene un préstamo o no

In [ ]:
#Vemos que tener un préstamo correlaciona de forma muy significativa con las variables:Ord_Loan_Payment,Num_Sym_LoanPayment y 
#Ord_Loan_Payment_amount
df_original.corr()["account_loan_bin"]

In [ ]:
#Vemos las correlaciones de las variables anteriores
print(np.corrcoef(df_original["account_loan_bin"],df_original["Ord_Loan_Payment"]))
print(np.corrcoef(df_original["account_loan_bin"],df_original["Ord_Loan_Payment_amount"]))
print(np.corrcoef(df_original["account_loan_bin"],df_original["Num_Sym_LoanPayment"]))

In [ ]:
#Vamos a ver qué resultados obtenemos eliminando la variable 'Num_Sym_LoanPayment'
X1=X.drop(['Num_Sym_LoanPayment'], axis=1)

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.2, random_state=0)

In [ ]:
X1_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_treeX1=X1_clf_tree.fit(X1_train,y1_train)

In [ ]:
#Aplicamos oversampling
smo=SMOTENC(categorical_features=range(45,58),random_state=0)# Modificamos rango porque hemos eliminado una variable
os_X1,os_y1=smo.fit_sample(X1_train, y1_train)
columns = X1_train.columns
os_X1 = pd.DataFrame(data=os_X1,columns=columns)
os_y1= pd.DataFrame(data=os_y1,columns=['account_loan_bin'])

In [ ]:
#Chequeamos que SMOTENC funciona como esperábamos

print("length of oversampled data is ",len(os_X1))
print("Number of loans=0 in oversampled data",len(os_y1[os_y1['account_loan_bin']==0]))
print("Number of loans=1",len(os_y1[os_y1['account_loan_bin']==1]))
print("Proportion of loans=0 is ",len(os_y1[os_y1['account_loan_bin']==0])/len(os_X1))
print("Proportion of loans=1 is ",len(os_y1[os_y1['account_loan_bin']==1])/len(os_X1))

os_bin=os_X1[['account_disponent_bin_0','account_disponent_bin_1',
       'frequency_After_trans', 'frequency_Monthly', 'frequency_Weekly',
       'sex_owner_F', 'sex_owner_M', 'owner_card_type_0', 'owner_card_type_1',
       'owner_card_type_2', 'owner_card_type_3', 'sex_disponent_F',
       'sex_disponent_M']]
print("unique de variables categóricas",unique(os_bin))
print("unique de variable y",unique(os_y1))

In [ ]:
#Vemos que el oversampling funciona bien, no lo vamos a chequear las próximas ocasiones

In [ ]:
osX1_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_tree_osX1=osX1_clf_tree.fit(os_X1,os_y1)

In [ ]:
#Evaluamos los modelos
print("Sin Oversampling")
eval_modelo(clf_treeX1,X1_train,y1_train, X1_test, y1_test)
print("Con Oversampling")
eval_modelo(clf_tree_osX1,os_X1,os_y1, X1_test, y1_test)

In [ ]:
dibu_arb(clf_treeX1)

In [ ]:
dibu_arb(clf_tree_osX1)

In [ ]:
X1_train.columns[37]

In [ ]:
X1.columns[clf_tree_osX1.feature_importances_>0.10]

In [ ]:
X1.columns[clf_treeX1.feature_importances_>0.10]

In [ ]:
#El modelo continua dando unos resultados espectaculares, vamos a ver que sucede si eliminamos la variable 
#'Ord_Loan_Payment_amount'

In [ ]:
X2=X1.drop(['Ord_Loan_Payment_amount'], axis=1)

In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, test_size=0.2, random_state=0)

In [ ]:
X2_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_treeX2=X2_clf_tree.fit(X2_train,y2_train)

In [ ]:
#Aplicamos oversampling
smo=SMOTENC(categorical_features=range(44,57),random_state=0)# Modificamos rango porque hemos eliminado una variable
os_X2,os_y2=smo.fit_sample(X2_train, y2_train)
columns = X2_train.columns
os_X2 = pd.DataFrame(data=os_X2,columns=columns)
os_y2= pd.DataFrame(data=os_y2,columns=['account_loan_bin'])

In [ ]:
osX2_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_tree_osX2=osX2_clf_tree.fit(os_X2,os_y2)

In [ ]:
#Evaluamos los modelos
print("Sin Oversampling")
eval_modelo(clf_treeX2,X2_train,y2_train, X2_test, y2_test)
print("Con Oversampling")
eval_modelo(clf_tree_osX2,os_X2,os_y2, X2_test, y2_test)

In [ ]:
dibu_arb(clf_treeX2)

In [ ]:
dibu_arb(clf_tree_osX2)

In [ ]:
X2_train.columns[8]

In [ ]:
X2.columns[clf_treeX2.feature_importances_>0.10]

In [ ]:
X2.columns[clf_tree_osX2.feature_importances_>0.10]

In [ ]:
#El modelo continua dando resultados espectaculares, vamos a ver que sucede si eliminamos la variable Ord_Loan_Payment

In [ ]:
X3=X2.drop(['Ord_Loan_Payment'], axis=1)

In [ ]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y, test_size=0.2, random_state=0)

In [ ]:
X3_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_treeX3=X3_clf_tree.fit(X3_train,y3_train)

In [ ]:
#Aplicamos oversampling
smo=SMOTENC(categorical_features=range(43,56),random_state=0)# Modificamos rango porque hemos eliminado una variable
os_X3,os_y3=smo.fit_sample(X3_train, y3_train)
columns = X3_train.columns
os_X3 = pd.DataFrame(data=os_X3,columns=columns)
os_y3= pd.DataFrame(data=os_y3,columns=['account_loan_bin'])

In [ ]:
osX3_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_tree_osX3=osX3_clf_tree.fit(os_X3,os_y3)

In [ ]:
#Evaluamos los modelos
print("Sin Oversampling")
eval_modelo(clf_treeX3,X3_train,y3_train, X3_test, y3_test)
print("Con Oversampling")
eval_modelo(clf_tree_osX3,os_X3,os_y3, X3_test, y3_test)

In [ ]:
#Con Oversampling estoy "forzando" al modelo a generar más positivos  (tp y fp), y en este caso se ve perfectamente como
# con oversampling mejoro el recall (porque tengo más positivos), pero empeoro la precision porque no los estoy prediciendo
#correctamente

In [ ]:
dibu_arb(clf_treeX3)

In [ ]:
dibu_arb(clf_tree_osX3)

In [ ]:
print(X3_train.columns[19])#Número de un tipo especial de reintegros
print(X3_train.columns[22])#Número de transferencias enviadas

In [ ]:
X3.columns[clf_treeX3.feature_importances_>0.10]

In [ ]:
X3.columns[clf_tree_osX3.feature_importances_>0.10]

In [ ]:
clf_treeX3.feature_importances_

In [ ]:
sum(clf_treeX3.feature_importances_)

In [ ]:
print(df_original.corr()["account_loan_bin"])

In [ ]:
#Después de eliminar las variables que más suenan a "préstamo", vamos a ver que sucede si eliminamos la variable Num_Type_VYBER,
#que es la de mayor importancia.Esta variable se corresponde con un tipo especial de reintegros

In [ ]:
X4=X3.drop(['Num_Type_VYBER'], axis=1)

In [ ]:
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y, test_size=0.2, random_state=0)

In [ ]:
X4_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_treeX4=X4_clf_tree.fit(X4_train,y4_train)

In [ ]:
#Aplicamos oversampling
smo=SMOTENC(categorical_features=range(42,55),random_state=0)# Modificamos rango porque hemos eliminado una variable
os_X4,os_y4=smo.fit_sample(X4_train, y4_train)
columns = X4_train.columns
os_X4 = pd.DataFrame(data=os_X4,columns=columns)
os_y4= pd.DataFrame(data=os_y4,columns=['account_loan_bin'])

In [ ]:
osX4_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)
clf_tree_osX4=clf_tree.fit(os_X4,os_y4)

In [ ]:
#Evaluamos los modelos
print("Sin Oversampling")
eval_modelo(clf_treeX4,X4_train,y4_train, X4_test, y4_test)
print("Con Oversampling")
eval_modelo(clf_tree_osX4,os_X4,os_y4, X4_test, y4_test)

In [ ]:
#Con los datos de X4 vemos que el oversampling genera el mismo efecto que en el caso de X3, mejora recall y empeora precision.
#Además, al eliminar la variable de 'Num_Type_VYBER' es destacable que el modelo que obtenemos tiende a generar un mayor número 
#de positivos, pero estos positivos resultan ser falsos positivos en su mayoría 

# ¿Puedo considerar que el oversampling no me está aportando nada?

In [ ]:
dibu_arb(clf_treeX4)

In [ ]:
dibu_arb(clf_tree_osX4)

In [ ]:
# Me quedo con el DataSet X3 no elimino la variable Num_Type_VYBER, ya que los resultados, aunque son peores, son ahora
# más razonables (teniendo)

In [ ]:
# Vamos a seguir con árboles ya que es un modelo sencillo y que podría aportar explicabilidad

In [ ]:
# Vamos a hacer scaling, aplicado a árboles no debería tener un gran impacto.

In [ ]:
#Vamos a probar Robust Scaling, partiendo del DataFrame X3 (después de eliminar las variables que parece que contenían 
#la información de si una cuenta ha contratado préstamo o no lo ha contratado)
from sklearn.preprocessing import RobustScaler
rbs = RobustScaler()
columns = X3.columns
rbs_scale = rbs.fit_transform(X3)
X=pd.DataFrame(rbs_scale,columns=columns)
X_rbs_sca=X.copy #Por si luego lo utilizamos
X.shape

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)#Con scaling
clf_tree=X_clf_tree.fit(X_train,y_train)

In [ ]:
#Comparamos los modelos con y sin scaling
print("Sin Scaling")
eval_modelo(clf_treeX3,X3_train,y3_train, X3_test, y3_test)
print("Con Scaling")
eval_modelo(X_clf_tree,X_train,y_train, X_test, y_test)

In [ ]:
# Los resultados con Robust Scaling no mejoran, así que vamos a probar con Standard Scaling aunque en un modelo de
# Decision Tree este modificación acostumbrará a tener poco impacto.

In [ ]:
from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
columns = X3.columns
scl_scale = scl.fit_transform(X3)
X=pd.DataFrame(scl_scale,columns=columns)
X_sta_sca=X.copy #Por si luego lo utilizamos
X.shape

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)#Con scaling
clf_tree_=X_clf_tree.fit(X_train,y_train)

In [ ]:
#Comparamos los modelos con y sin scaling
print("Sin Scaling")
eval_modelo(clf_treeX3,X3_train,y3_train, X3_test, y3_test)
print("Con Scaling")
eval_modelo(clf_tree,X_train,y_train, X_test, y_test)

In [ ]:
#Vamos a probar si con el scaling, cuando hacemos PCA podemos obtener una representación simple de los datos. 
#Sé que va a ser difícil pero lo probamos.
pca = PCA(n_components=2).fit_transform(X)
plt.figure(dpi=120)
plt.scatter(pca[y.values==0,0], pca[y.values==0,1], alpha=0.5, label='NO', s=2, color='navy')
plt.scatter(pca[y.values==1,0], pca[y.values==1,1], alpha=0.5, label='YES', s=2, color='darkorange')
plt.legend()
plt.title('Producto préstamo')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
#Del gráfico anterior no consigo sacar nada en claro
pca2 = PCA(n_components=2)
pca2.fit(X)
print(pca2.components_)
print(pca2.explained_variance_ratio_)

In [ ]:
#De las componentes principales de momento tampoco sacamos ninguna conclusión, pero al haber escalado los datos ahora obtenemos un dato 
#más razonable de variabilidad explicada

In [ ]:
#Standard Scaling tampoco ha funcionado mejor que sin scaling. Aplicamos scaling sólo a las variables numéricas
print(X3.columns)
print(X3.shape)

In [ ]:
#Separamos las variables numéricas de las que consideramos categóricas
X3_cat=X3[['account_disponent_bin_0','account_disponent_bin_1',
       'frequency_After_trans', 'frequency_Monthly', 'frequency_Weekly',
       'sex_owner_F', 'sex_owner_M', 'owner_card_type_0', 'owner_card_type_1',
       'owner_card_type_2', 'owner_card_type_3', 'sex_disponent_F',
       'sex_disponent_M']]
X3_num=X3[['Date_Account', 'birth_owner', 'birth_disponent', 'owner_card_date',
       'Ord_Insurance', 'Ord_Insurance_amount', 'Ord_Household_Payment',
       'Ord_Household_Payment_amount', 'Ord_Leasing', 'Ord_Empty',
       'Ord_Empty_amount', 'num_inhabitants', 'municip < 499',
       'municip 500-1999', 'municip 2000-9999', 'municip > 10000',
       'num_cities', 'avg_salary', 'Num_Type_Credit', 'Num_Type_VYBER',
       'Num_Type_Withdrawal', 'Num_Op_Null', 'Num_Op_Remittances',
       'Num_Op_Collection', 'Num_Op_CashCredit', 'Num_Op_WithdrawalCash',
       'Num_Op_WithdrawalCreditCard', 'Num_Sym_Null', 'Num_Sym_Null2',
       'Num_Sym_Pension', 'Num_Sym_Insurance', 'Num_Sym_NegBal',
       'Num_Sym_Household', 'Num_Sym_Statement', 'Num_Sym_IntDep',
       'Balance_in_negative', 'Ord_Leasing_amount', 'ratio_urban_inhabitants',
       'unemployment_rate_95', 'unemployment_rate_96', 'crimes_95_ratio',
       'crimes_96_ratio', 'entrepreneurs_ratio']]

In [ ]:
#Probamos con el Robust Scaling
columns = X3_num.columns
X3_scale = rbs.fit_transform(X3_num)
X3_scale=pd.DataFrame(X3_scale,columns=columns)
X = pd.concat([X3_scale,X3_cat], axis = 1)
X.shape
X_sca_num=X.copy #Guardamos este DataFrame por si la utilizamos posteriormente

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)#Con scaling
clf_tree_=X_clf_tree.fit(X_train,y_train)

In [ ]:
#Comparamos los modelos con y sin scaling
print("Sin Scaling")
eval_modelo(clf_treeX3,X3_train,y3_train, X3_test, y3_test)
print("Con Robust Scaling sólo en variables numéricas")
eval_modelo(clf_tree,X_train,y_train, X_test, y_test)

In [ ]:
#En este caso, el robust scaling tampoco mejora los resultados que teníamos originalmente. Probamos otra alternativa

In [ ]:
#Probamos con min-max scaling
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()

In [ ]:
columns = X3_num.columns
X3_scale = minmax.fit_transform(X3_num)
X3_scale=pd.DataFrame(X3_scale,columns=columns)
X = pd.concat([X3_scale,X3_cat], axis = 1)
X.shape
Xminmax=X.copy

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=5,random_state=0)#Con scaling
clf_tree_=X_clf_tree.fit(X_train,y_train)

In [ ]:
#Comparamos los modelos con y sin scaling
print("Sin Scaling")
eval_modelo(clf_treeX3,X3_train,y3_train, X3_test, y3_test)
print("Con Scaling")
eval_modelo(clf_tree,X_train,y_train, X_test, y_test)

In [ ]:
#Con scaling MinMax el modelo casi no genera predicciones positivas y el accuracy se obtiene por un modelo que básicamente
#predice en test que la cuenta no contrata préstamo y al estar las clases desbalanceadas obtenemos ese acuraccy, pero muy mal
#recall y precision

# Hasta el momento hemos visto que en el dataset que habíamos generado 
habían 3 variables que "contenían" la información de la variable que queríamos predecir y también hemos visto que, en el caso de Decision Trees, las técnicas de Oversampling y de Scaling tal y como las hemos aplicado no mejoran los modelos iniciales.

# A continuación trabajamos sobre las features numéricas

In [ ]:
print(X3_num.columns)

In [ ]:
#A partir del DataFrame X3 creamos un DataFrame que contenga el log natural de las variables numéricas
cols = X3_num.columns
X3_log=pd.DataFrame(X3_num,columns=columns)#También podríamos hacer un copy del DF

In [ ]:
#Calculamos el logaritmo de las variables que consideramos numéricas
cols=X3_num.columns
for col in cols:
    X3_log[col]=X3_log[col]+1.1 #Para evitar negativos al aplicar el logaritmo añadimos 1.1
    X3_log[col]=np.log(X3_log[col])

In [ ]:
X3_log.head()

In [ ]:
#Hemos conseguido diminuir la variabilidad de las features
X3_log.describe()

In [ ]:
#Creamos un DataFrame a partir del DataFrame X3 (una vez eliminadas las 3 variables que contenían la información de la variable
#a predecir), aplicando logaritmo a las variables que consideramos numéricas y dejando las categóricas igual.
X_log= pd.concat([X3_log,X3_cat], axis = 1)

In [ ]:
X_log.head()

In [ ]:
#Vamos a crear un nuevo DataFrame binarizando las variables que tienen el número de operaciones (pasan de tener el número de 
#operaciones a decir si la cuenta ha realizado ese tipo de operativa o no) y eliminando las columnas que contienen
#el sufijo "_amount" (sólo queremos reflejar si se ha realizado un tipo de operativa o no se ha realizado)

col_to_bin=['Ord_Insurance', 'Ord_Household_Payment','Ord_Leasing', 'Ord_Empty','Num_Type_Credit', 'Num_Type_VYBER',
       'Num_Type_Withdrawal', 'Num_Op_Null', 'Num_Op_Remittances','Num_Op_Collection', 'Num_Op_CashCredit', 
       'Num_Op_WithdrawalCash','Num_Op_WithdrawalCreditCard', 'Num_Sym_Null', 'Num_Sym_Null2',
       'Num_Sym_Pension', 'Num_Sym_Insurance', 'Num_Sym_NegBal','Num_Sym_Household', 'Num_Sym_Statement', 'Num_Sym_IntDep',
       'Balance_in_negative']

col_out=['Ord_Insurance_amount','Ord_Household_Payment_amount','Ord_Empty_amount', 'Ord_Leasing_amount']

In [ ]:
cols = X3.columns
X3_bin=pd.DataFrame(X3,columns=cols)
X3_bin=X3_bin.drop(col_out, axis=1)#Eliminamos las columnas de amount
X3_bin=X3_bin.drop(col_to_bin, axis=1)#Eliminamos las columnas a binarizar

In [ ]:
X3_bin.shape

In [ ]:
#Definimos el dataset que queremos binarizar
X3_to_bin=X3[['Ord_Insurance', 'Ord_Household_Payment','Ord_Leasing', 'Ord_Empty','Num_Type_Credit', 'Num_Type_VYBER',
       'Num_Type_Withdrawal', 'Num_Op_Null', 'Num_Op_Remittances','Num_Op_Collection', 'Num_Op_CashCredit', 
       'Num_Op_WithdrawalCash','Num_Op_WithdrawalCreditCard', 'Num_Sym_Null', 'Num_Sym_Null2',
       'Num_Sym_Pension', 'Num_Sym_Insurance', 'Num_Sym_NegBal','Num_Sym_Household', 'Num_Sym_Statement', 'Num_Sym_IntDep',
       'Balance_in_negative']]

In [ ]:
def binario(x):
    if x>0:
        x=1
    else:
        x=0
    return x

In [ ]:
for col in col_to_bin:
    X3_to_bin[col]=X3_to_bin[col].apply(binario)

In [ ]:
X3_to_bin=X3_to_bin.astype(str)
X3_to_bin=pd.get_dummies(X3_to_bin)

# ¿Porque hay algunas columnas que no se duplican con el get_dummies?

In [ ]:
#Creamos un DataFrame a partir del DataFrame X3 (una vez eliminadas las 3 variables que conteníanla información de la variable
#dependiente), binarizando las variables que contaban el número de operaciones.
X_bin= pd.concat([X3_bin,X3_to_bin], axis = 1)

In [ ]:
X_bin.shape

In [ ]:
df_original.head()

# Con los nuevos DataFrames de features vamos a intentar encontrar el mejor Decision Tree y ver cuales son las variables más significativas 

In [ ]:
#Los DataFrames con los que vamos a trabajar son:
#X_orig: El Dataframe con el que habíamos trabajado antes eliminando 3 variables, también lo llamábamos X3.
X=X3
#X_log: Es el Dataframe X_orig pero haciendo el logaritmo natural a las variables numéricas
#X_bin: Es el Dataframe X_orig pero binarizando las variables que contaban cuantas operaciones de cada tipo se habían realizado
#en cada cuenta, ahora decimos si la cuenta tiene ese tipo de operativa o no, y eliminando las variables que cuantificaban los
#importes de dicha operativa

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#Inicialmente pensaba considerar como única métrica la precisión (quería que el modelo minimizase los falsos positivos,
#pero en vista de los resultados de los modelos anteriores, en los que el recall (el % de positivos que acierto) 
#puede ser muy bajo voy a comenzar a monitorizar también el F1 Score 

In [ ]:
#Incluimos la métrica del F1 Score en la evaluación de los modelos
def eval_modelo_2 (clf,X_train,y_train, X_test,y_test):
    print("Datos de train:")
    print("El accuracy es",accuracy_score(y_train,clf.predict(X_train))*100,"%")
    print("La precision es",precision_score(y_train,clf.predict(X_train))*100, "%")
    print("El recall es",recall_score(y_train,clf.predict(X_train))*100, "%")
    print("El F1 Score es",f1_score(y_train,clf.predict(X_train))*100,"%")
    tn, fp, fn, tp=confusion_matrix(y_train,clf.predict(X_train)).ravel()
    print("tn:",tn," fp:",fp," fn:",fn," tp:",tp)
    print("Datos de test:")
    print("El accuracy es",accuracy_score(y_test,clf.predict(X_test))*100,"%")
    print("La precision es",precision_score(y_test,clf.predict(X_test))*100, "%")
    print("El recall es",recall_score(y_test,clf.predict(X_test))*100,"%")
    print("El F1 Score es",f1_score(y_test,clf.predict(X_test))*100,"%")
    tn_t, fp_t, fn_t, tp_t=confusion_matrix(y_test,clf.predict(X_test)).ravel()
    print("tn:",tn_t," fp:",fp_t," fn:",fn_t," tp:",tp_t)      

In [ ]:
#Vamos a definir la parrilla para realizar Randomized Grid Search

# Máximo número de niveles en el árbol. Damos una distribución con mayor probabilidad en valores pequeños
max_depth1 = [int(x) for x in np.linspace(2, 20, num = 10)]#"Sobreponderamos" árboles con poca profundidad
max_depth2 =[int(x) for x in np.linspace(30, 100, num = 4)]
max_depth=max_depth1 + max_depth2

# Mínimo número de observaciones en cada hoja.Damos una distribución con mayor probabilidad en valores pequeños
min_samples_leaf_1 = [int(x) for x in np.linspace(5, 50, num = 4)]
min_samples_leaf_2 = [int(x) for x in np.linspace(60, 100, num = 10)]
min_samples_leaf=min_samples_leaf_1+min_samples_leaf_2


In [ ]:
max_depth

In [ ]:
min_samples_leaf

In [ ]:
# Creamos la grid aleatoria
random_grid = {'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf
               }

In [ ]:
random_grid

In [ ]:
clf_tree = DecisionTreeClassifier(random_state=0)
#Creamos una versión para optimizar la precision
clf_tree_random_p= RandomizedSearchCV(random_state=0,estimator = clf_tree, param_distributions = random_grid, n_iter = 100, cv = 5,scoring="precision")
#Creamos una versión para optimizar el F1 Score
clf_tree_random_f1= RandomizedSearchCV(random_state=0,estimator = clf_tree, param_distributions = random_grid, n_iter = 100, cv = 5,scoring="f1")

In [ ]:
#Comenzamos por el DataFrame X (el X3 anterior)
#Generamos conjuntos de train y el de test. Para el test usamos el 20% de las observaciones
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
clf_tree_random_p.fit(X_train, y_train)

In [ ]:
print("tuned hpyerparameters :(best parameters) ",clf_tree_random_p.best_params_)

In [ ]:
#Trabajamos ahora con el mejor modelo encontrado en el Randomized Search, optimizando la precision. Entrenamos ahora el modelo
#con todos los datos de train y luego lo evaluaremos con un conjunto de test no utilizado en la estimación
clf_tree_p = clf_tree_random_p.best_estimator_
clf_tree_p.fit(X_train,y_train)

In [ ]:
eval_modelo_2 (clf_tree_p,X_train,y_train, X_test,y_test)

In [ ]:
#Parece que el modelo anterior produce overfitting

In [ ]:
X.columns[clf_tree_p.feature_importances_>0.10]
#Las variables que obtenemos de este árbol que parece que produce overfitting son las que habíamos visto anteriormente 

In [ ]:
#Vemos ahora los resultados con la optimización realizada sobre F1 Score

clf_tree_random_f1.fit(X_train, y_train)

print("tuned hpyerparameters :(best parameters) ",clf_tree_random_f1.best_params_)

clf_tree_f1 = clf_tree_random_f1.best_estimator_  
clf_tree_f1.fit(X_train,y_train)

eval_modelo_2 (clf_tree_f1,X_train,y_train, X_test,y_test)

In [ ]:
#Atención: obtengo mejor precision en el modelo que optimiza F1 Score que en el que optimiza Precision. Dado que el coste  
#computacional es bajo, creo que va a ser mejor aplicar GridSearch CV en lugar de RandomizedGridSearchCV, para no obtener 
#resultados incoherentes

In [ ]:
#Comparando el modelo que optimizamos el F1 Score, respecto al que optimizamos la precision, creo que es mejor el que optimiza
#el F1 Score porque la precision puede bajar un poco, el recall aumenta de forma más significativa. Esto lo consigue 
#generando un mayor número de positivos

In [ ]:
#Tomamos el árbol anterior como árbol de referencia ya que ofrece el mejor balance de precision-recall
clf_tree_best=clf_tree_f1

In [ ]:
X.columns[clf_tree_f1.feature_importances_>0.10]

In [ ]:
#Parece que los modelos anteriores overfittean. Vamos a probar un modelo más sencillo
clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=3,random_state=0)
clf_tree.fit(X_train,y_train)
eval_modelo_2(clf_tree,X_train,y_train, X_test,y_test)

In [ ]:
clf_tree_benchmark=clf_tree

In [ ]:
dibu_arb(clf_tree_benchmark)

In [ ]:
print(X_train.columns[19])
print(X_train.columns[25])
print(X_train.columns[22])
print(X_train.columns[24])
print(X_train.columns[36])

In [ ]:
#Interpretando el árbol anterior, lo que obtenemos es que, mayoritariamente, las cuentas que contratan préstamos son las que:
#--Num_Type_Viber > 0.5 (reintegros del tipo especial Vyber)
#--Num_Op_Remittances > 0.5 (envios/transferencias a otros bancos). Este podría ser un indicador de la solvencia y por tanto
#le encuentro sentido a su aparición
#--Ord_Leasing Amount<=501,5 (cargos automáticos por Leasing)

In [ ]:
dibu_arb(clf_tree_best)

In [ ]:
X.columns[clf_tree_f1.feature_importances_>0.10]

In [ ]:
#plot_importance(clf_tree_f1)
#pyplot.show()

In [ ]:
#De momento parece que la información aportada por los modelos obtenidos con randomized search y un modelo sencillo con
#criterio experto aportan resultados bastante similares

In [ ]:
#Comenzamos por el DataFrame X_log
#Generamos conjuntos de train y de test. Para el test usamos el 20% de las observaciones
X_log_train, X_log_test, y_log_train, y_log_test = train_test_split(X_log, y, test_size=0.2, random_state=0)

In [ ]:
#Vamos a seguir trabajando con árboles por lo que, como en el caso del scaling, no esperamos mejoras significativas

In [ ]:
clf_tree_random_p.fit(X_log_train, y_log_train)
print("tuned hpyerparameters :(best parameters) ",clf_tree_random_p.best_params_)

In [ ]:
clf_tree_p = clf_tree_random_p.best_estimator_  
clf_tree_p.fit(X_log_train,y_log_train)
eval_modelo_2(clf_tree_p,X_log_train, y_log_train,X_log_test, y_log_test)

In [ ]:
X.columns[clf_tree_p.feature_importances_>0.10]

In [ ]:
#Optimizamos ahora el F1 Score
clf_tree_random_f1.fit(X_log_train, y_log_train)
print("tuned hpyerparameters :(best parameters) ",clf_tree_random_f1.best_params_)

In [ ]:
clf_tree_f1 = clf_tree_random_f1.best_estimator_  
clf_tree_f1.fit(X_log_train, y_log_train)
eval_modelo_2 (clf_tree_f1,X_log_train, y_log_train,X_log_test, y_log_test)

In [ ]:
#Atención: obtengo mejor precision en el modelo que optimiza F1 Score que en el que optimiza Precision. Dado que el coste  
#computacional es bajo, creo que va a ser mejor aplicar GridSearch CV en lugar de RandomizedGridSearchCV, para no obtener 
#resultados incoherentes

In [ ]:
X_log.columns[clf_tree_f1.feature_importances_>0.10]

In [ ]:
#Parece que los modelos anteriores overfittean. Vamos a probar un modelo más sencillo
clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=3,random_state=0)
clf_tree.fit(X_log_train, y_log_train)
eval_modelo_2(clf_tree,X_log_train, y_log_train,X_log_test, y_log_test)

In [ ]:
X_log.columns[clf_tree_f1.feature_importances_>0.10]

In [ ]:
#Vemos que hemos obtenido features muy similares probando con técnicas más complejas como Randomized Search y con 
#criterio experto o lo que podríamos haber obtenido a partir de un modelo benchmark inicial

In [ ]:
#Seguimos con el DataFrame X_bin, en el que hemos binarizado las variables de 
#Generamos conjuntos de train y de test. Para el test usamos el 20% de las observaciones
X_bin_train, X_bin_test, y_bin_train, y_bin_test = train_test_split(X_bin, y, test_size=0.2, random_state=0)

In [ ]:
clf_tree_random_p.fit(X_bin_train, y_bin_train)
print("tuned hpyerparameters :(best parameters) ",clf_tree_random_p.best_params_)

In [ ]:
clf_tree_p = clf_tree_random_p.best_estimator_  
clf_tree_p.fit(X_bin_train,y_bin_train)
eval_modelo_2(clf_tree_p,X_bin_train, y_bin_train,X_bin_test, y_bin_test)

In [ ]:
X_bin.columns[clf_tree_p.feature_importances_>0.10]

In [ ]:
#Optimizamos ahora el F1 Score
clf_tree_random_f1.fit(X_bin_train, y_bin_train)
print("tuned hpyerparameters :(best parameters) ",clf_tree_random_f1.best_params_)

In [ ]:
clf_tree_f1 = clf_tree_random_f1.best_estimator_  
clf_tree_f1.fit(X_bin_train, y_bin_train)
eval_modelo_2 (clf_tree_f1,X_bin_train, y_bin_train,X_bin_test, y_bin_test)

In [ ]:
#Con este DataFrame, los resultados son claramente los peores

In [ ]:
X_bin.columns[clf_tree_f1.feature_importances_>0.10]

In [ ]:
#Los DataFrames aplicando logaritmo y binarizando no mejoran el resultado del DataFrame original, se puede entender porque el
#modelo que estamos aplicando es un árbol.

#También estamos viendo que consistentemente las variables que son más relevantes son:
#--Num_Type_VYBER
#--Num_Op_Remittances
#--Ord_Household_Payment_amount
#--Ord_Leasing

#A continuación vamos a aplicar GridSearch con el conjunto de datos procedente de X3 (datos originales menos 3 variables)
#y vamos a ver que variables nos salen como más relevantes

In [ ]:
#Utilizamos la misma matriz de hiperparámetros
grid=random_grid

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
k=StratifiedKFold(n_splits=5, random_state=0,shuffle=False)

In [ ]:
#Creamos los modelos para hacer la búsqueda de hiperparámetros 
clf_tree = DecisionTreeClassifier(random_state=0)
#Creamos una versión para optimizar la precision
clf_tree_gridsearch_p= GridSearchCV(estimator = clf_tree, param_grid = grid, cv=k, scoring="precision")
#Creamos una versión para optimizar el F1 Score
clf_tree_gridsearch_f1= GridSearchCV(estimator = clf_tree, param_grid = grid, cv=k, scoring="f1")

In [ ]:
#Vemos ahora los resultados con la optimización realizada sobre Precision

clf_tree_gridsearch_p.fit(X_train, y_train)

print("tuned hpyerparameters :(best parameters) ",clf_tree_gridsearch_p.best_params_)

clf_tree_gs_p = clf_tree_gridsearch_p.best_estimator_  
clf_tree_gs_p.fit(X_train,y_train)

eval_modelo_2 (clf_tree_gs_p,X_train,y_train, X_test,y_test)

In [ ]:
#Veamos las features de mayor relvancia en el modelo
X.columns[clf_tree_gs_p.feature_importances_>0.10]

In [ ]:
#Vemos ahora los resultados con la optimización realizada sobre F1 Score

clf_tree_gridsearch_f1.fit(X_train, y_train)

print("tuned hpyerparameters :(best parameters) ",clf_tree_gridsearch_f1.best_params_)

clf_tree_gs_f1 = clf_tree_gridsearch_f1.best_estimator_
clf_tree_gs_f1.fit(X_train,y_train)

eval_modelo_2 (clf_tree_gs_f1,X_train,y_train, X_test,y_test)

¿Cómo puede ser que obtenga un mejor resultado en Precision cuando optimizo F1 que cuando optimizo precision?. Creo que tengo 
todos los Random State controlados. Entonces puede ser que al volver  entrenar el modelo con el conjunto completo de los datos
de train, cambie el modelo y su evaluación.
Aunque parece que el modelo produce overfitting, de momento vamos a considerar este modelo para realizar las interpretaciones, ya que en datos de test es el que mejor resultados muestra.

In [ ]:
#Veamos la relevancia de las features en el modelo
clf_tree_gs_f1.feature_importances_

In [ ]:
#Vemos que muchas features tienen relevancia cero  o prácticamente nula 
#Veamos las features de mayor relevancia en el modelo
X.columns[clf_tree_gs_f1.feature_importances_>0.10]

# Analizamos a continuación la interpretabilidad de los resultados obtenidos 

In [ ]:
#Para ver el impacto de las variables en la probabilidad de que una cuenta tenga un préstamo vamos a aplicar Partial Dependence

In [ ]:
#pip install pdpbox

In [ ]:
from pdpbox import pdp

In [ ]:
#Ruta interesante para entender el Partial Dependece Plot
#https://christophm.github.io/interpretable-ml-book/pdp.html

In [ ]:
#También vamos a utilizar Shap Values

In [ ]:
pip install shap==0.23.0

In [ ]:
#pip install -I shap

In [ ]:
import shap
shap.initjs()

In [ ]:
#Comenzamos el análisis de los SHAP values
explainer = shap.TreeExplainer(clf_tree_gs_f1)

In [ ]:
#Si intentamos realizar el análisis de shap para el conjunto de Train, nos dice que son demasiadas observaciones a considerar.
#Nos recomienda que no tomemos más de 3.000. Tomamos 2.900 de los 3.600 que tenemos en total.
X_train_random=X_train.sample(n=2900, random_state=0)
shap_values_train = explainer.shap_values(X_train_random)

In [ ]:
#Preparamos tambien los shap values para el test
shap_values_test = explainer.shap_values(X_test)

# Análisis de variable Num_Type_VYBER

In [ ]:
#Vemos el impacto de 'Num_Type_VYBER' en X_train con Partial Dependece Plot
pdp_goals = pdp.pdp_isolate(model=clf_tree_gs_f1, dataset=X_train, model_features=X.columns, feature='Num_Type_VYBER')
pdp.pdp_plot(pdp_goals, 'Num_Type_VYBER')
plt.show()

In [ ]:
#Vemos el impacto de 'Num_Type_VYBER' en X_test con Partial Dependece Plot
pdp_goals = pdp.pdp_isolate(model=clf_tree_gs_f1, dataset=X_test, model_features=X.columns, feature='Num_Type_VYBER')
pdp.pdp_plot(pdp_goals, 'Num_Type_VYBER')
plt.show()

Para el caso de Num_Type_VYBER, vemos que el tener este tipo de operativa incrementa la posibilidad de haber contratado
un préstamo hasta en un 20%, aproximadamente

In [ ]:
#Veamos la distribución de la variable y en cuantas observaciones tiene efecto esta feature
plt.hist(X['Num_Type_VYBER'])

In [ ]:
X[X['Num_Type_VYBER']>0].shape[0]

In [ ]:
#Para asegurar la consistencia del análisis de Partial Dependence vamos a analizar la correlación de esta variable con el resto
#de variables
X.corr()["Num_Type_VYBER"]

Vemos que "Num_Type_VYBER" tiene correlación que podría ser elevada con "Num_Sym_Null" y "Num_Op_WithdrawalCash", entorno
a un 0,50.

Vamos a ver que conclusiones podemos obtener con SHAP Values 

In [ ]:
#SHAP Values para  conjunto de train. En los desplegables de X e Y hay que seleccionar la variable Num_Type_VYBER
shap.force_plot(explainer.expected_value[1], shap_values_train[1], X_train_random)

In [ ]:
#SHAP Values para  conjunto de test. En los desplegables de X e Y hay que seleccionar la variable Num_Type_VYBER
shap.force_plot(explainer.expected_value[1], shap_values_test[1], X_test)

Vemos que en el caso de SHAP values, la relación entre el número de operaciones de tipo Vyber y su contribución a la probabilidad de contratar un préstamo no es monotónico. En general, para valores bajos de Num_Type_VYBER, esta característica
incrementa la posibilidad de haber contratado préstamo, pero para valores elevados de esta variable hay parece contribuir negativamente. 

El scatter plot que se muestra a continuación también muestra que las cuentas con mayor operativa de tipo Vyber, tienden a no haber contratado préstamos. Esta casuísitica debería analizarse en mayor profundidad, revisando dichas cuentas y viendo por ejemplo si contratan otro tipo de productos de crédito que no estén entrando en el fichero de loans.

Vemos si podrían ser los leasings, pero parece que las cuentas con mayor operativa de tipo Vyber, tienden a no tener domiciliaciones por leasings.

In [ ]:
plt.scatter(X.Num_Type_VYBER, y)
plt.xlabel("Num_Type_VYBER")
plt.ylabel("account_loan_bin")
plt.show()

In [ ]:
plt.scatter(X.Num_Type_VYBER, X.Ord_Leasing)
plt.xlabel("Num_Type_VYBER")
plt.ylabel("Ord_Leasing")
plt.show()

# Análisis de variable Num_Op_Remittances


In [ ]:
#Vemos el impacto de 'Num_Op_Remittances' en X_train con Partial Dependece Plot
pdp_goals = pdp.pdp_isolate(model=clf_tree_gs_f1, dataset=X_train, model_features=X.columns, feature='Num_Op_Remittances')
pdp.pdp_plot(pdp_goals, 'Num_Op_Remittances')
plt.show()

In [ ]:
#Vemos el impacto de 'Num_Op_Remittances' en X_test con Partial Dependece Plot
pdp_goals = pdp.pdp_isolate(model=clf_tree_gs_f1, dataset=X_test, model_features=X.columns, feature='Num_Op_Remittances')
pdp.pdp_plot(pdp_goals, 'Num_Op_Remittances')
plt.show()

Para el caso de Num_Op_Remittances, vemos que el tener este tipo de operativa puede incrementar la posibilidad 
de haber contratado un préstamo hasta en más del 50%

In [ ]:
#Veamos la distribución de la variable y en cuantas observaciones tiene efecto esta feature
plt.hist(X['Num_Op_Remittances'])

In [ ]:
X[X['Num_Op_Remittances']>0].shape[0]

In [ ]:
#Para asegurar la consistencia del análisis de Partial Dependence vamos a analizar la correlación de esta variable con el resto
#de variables
X.corr()["Num_Op_Remittances"]

Vemos que "Num_Op_Remittances" tiene correlación que podría ser elevada con "Num_Sym_Null2" (0,85), y "Num_Type_Withdrawal" (0,83) y condependence otras como "Num_Sym_Household" y "Ord_Insurance". Estas elevadas correlaciones pueden invalidar los resultados del análisis de Partial Dependence.

Vamos a ver que conclusiones podemos obtener con SHAP Values 

In [ ]:
#SHAP Values para  conjunto de train. En los desplegables de X e Y hay que seleccionar la variable Num_Op_Remittances
shap.force_plot(explainer.expected_value[1], shap_values_train[1], X_train_random)

In [ ]:
#SHAP Values para  conjunto de test. En los desplegables de X e Y hay que seleccionar la variable Num_Op_Remittances
shap.force_plot(explainer.expected_value[1], shap_values_test[1], X_test)

In [ ]:
plt.scatter(X.Num_Op_Remittances, y)
plt.xlabel("Num_Op_Remittances")
plt.ylabel("account_loan_bin")
plt.show()

Un número bajo de opdraciones de Remittances/Envíos de dinero no parece implicar mayor probabilidad de contratar un préstamo (es normal ya que la mayoría de cuentas tienen un número bajo de Remittances y no tienen préstamo). En cambio un número elevado de Remittances sí que implica una mayor probabilidad de haber contratado un préstamo.

En mi opinión lo anterior tiene 2 lecturas, realizar un gran número de envíos, puede ser una muestra de calidad crediticia y por tanto de ser elegible para obtener un crédito. Pero también podría guardar relación con que la cuenta haya cotnratado un préstamo y sea la forma de pago (a pesar de haber eliminado inicialmente las variables: Ord_Loan_Payment,Num_Sym_LoanPayment y 
Ord_Loan_Payment_amount). En consecuencia se debería profundizar en la finalidad de las operaciones de Remittance.

# Análisis de variable Ord_Leasing

In [ ]:
#Vemos el impacto de 'Ord_Leasing' en X_train con Partial Dependece Plot
pdp_goals = pdp.pdp_isolate(model=clf_tree_gs_f1, dataset=X_train, model_features=X.columns, feature='Ord_Leasing')
pdp.pdp_plot(pdp_goals, 'Ord_Leasing')
plt.show()

In [ ]:
pdp_goals = pdp.pdp_isolate(model=clf_tree_gs_f1, dataset=X_test, model_features=X.columns, feature='Ord_Leasing')
pdp.pdp_plot(pdp_goals, 'Ord_Leasing')
plt.show()

Para el caso de Ord_Leasing, vemos que el tener este tipo de operativa disminuye la probabilidad de haber contratado un préstamo en un 10%.

In [ ]:
#Veamos la distribución de la variable y en cuantas observaciones tiene efecto esta feature
plt.hist(X['Ord_Leasing'])

In [ ]:
X[X['Ord_Leasing']>0].shape[0]

In [ ]:
#Para asegurar la consistencia del análisis de Partial Dependence vamos a analizar la correlación de esta variable con el resto
#de variables
X.corr()["Ord_Leasing"]

Vemos que "Ord_Leasing" tiene correlación elevada con "Ord_Leasing_amount " (0,88) ya ambas son distintas de cero en los mismos casos y a medida que una de ellas crece la otra también debería hacerlo. Y la siguiente mayor correlación es con "Num_Type_VYBER" de 0,24. Si vemos es scatter plot de "Ord_Leasing" y "Num_Type_VYBER", vemos que valores altos de "Num_Type_VYBER" que "perjudican" la probabilidad de haber contratado un préstamo acostumbran a tener "Ord_Leasing"=0, que es un valor que no "perjudica" la probabilidad de haber contratado un préstamo y por tanto no podemos asumir que ambas variables contengan la misma información

A continuación, vamos a ver que conclusiones podemos obtener con SHAP Values: 

In [ ]:
plt.scatter(X.Num_Type_VYBER, X.Ord_Leasing)
plt.xlabel("Num_Type_VYBER")
plt.ylabel("Ord_Leasing")
plt.show()

In [ ]:
#SHAP Values para  conjunto de train. En los desplegables de X e Y hay que seleccionar la variable Ord_Leasing
shap.force_plot(explainer.expected_value[1], shap_values_train[1], X_train_random)

In [ ]:
#SHAP Values para  conjunto de test. En los desplegables de X e Y hay que seleccionar la variable Ord_Leasing
shap.force_plot(explainer.expected_value[1], shap_values_test[1], X_test)

In [ ]:
plt.scatter(X.Ord_Leasing, y)
plt.xlabel("Ord_Leasing")
plt.ylabel("account_loan_bin")
plt.show()

Vemos que tener Ord_Leasing=0, mejora ligeramente la probabilidad de haber contratado un préstamo, mientras que Ord_Leasing=1 disminuye dicha probabilidad.

Es también llamativo que ninguna de las cuentas que ha contratado un préstamo ha tenido activo las órdenes de Leasing (scatter plot). 

In [ ]:
#shap.TreeExplainer?

In [ ]:
#shap.force_plot?

In [ ]:
i=10
shap.force_plot(explainer.expected_value[1], shap_values[1][i,:], feature_names=X_train.columns)

In [ ]:
X_train.iloc[i][['Num_Type_VYBER','Num_Op_Remittances','Ord_Leasing']]

In [ ]:
explainer.expected_value[0]

In [ ]:
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.TreeExplainer(clf_tree_gs_f1).shap_interaction_values(X_test)

In [ ]:
#La obs 1 y 52 dan valor 0

In [ ]:
#Para interpretar Shap Values
#http://www.f1-predictor.com/model-interpretability-with-shap/

In [ ]:
X_test.iloc[0][['Num_Type_VYBER','Num_Op_Remittances','Ord_Leasing']]

In [ ]:
shap.force_plot?

In [ ]:
#Vamos a probar con TreeInterpreter

In [ ]:
#pip install treeinterpreter

In [ ]:
from treeinterpreter import treeinterpreter as ti

In [ ]:
prediction, bias, contributions = ti.predict(clf_tree_gs_f1, X_test)

In [ ]:
prediction, bias, contributions = ti.predict(clf_tree_gs_f1, X_test)
print ("Prediction", prediction)
print ("Bias (trainset prior)", bias)
print ("Feature contributions:")
for c, feature in zip(contributions[0], 
                             X_test.columns):
    print (feature, c)

In [ ]:
contributions

In [ ]:
#assert(np.allclose(prediction, bias + np.sum(contributions, axis=1)))

In [ ]:
#assert(np.allclose(clf_tree_gs_f1.predict(X_test), bias + np.sum(contributions, axis=1)))

In [ ]:
type(pdp_goals)

# Vamos a realizar el análisis ahora con modelos para los que no hay explicabilidad directa. Voy a probar con Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

#Vamos a definir la parrilla para realizar Randomized Grid Search

# Máximo número de niveles en el árbol. Damos una distribución con mayor probabilidad en valores pequeños
max_depth1 = [int(x) for x in np.linspace(2, 20, num = 10)]
max_depth2 =[int(x) for x in np.linspace(30, 100, num = 4)]
max_depth=max_depth1 + max_depth2
#max_depth.append(None)

# Mínimo número de observaciones en cada hoja.Damos una distribución con mayor probabilidad en valores pequeños
min_samples_leaf_1 = [int(x) for x in np.linspace(5, 50, num = 10)]
min_samples_leaf_2 = [int(x) for x in np.linspace(60, 100, num = 5)]
min_samples_leaf=min_samples_leaf_1+min_samples_leaf_2
#min_samples_leaf.append(None)

# Creamos la grid aleatoria
random_grid = {'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf,
               }

clf_tree = DecisionTreeClassifier()
clf_tree_random_p= RandomizedSearchCV(estimator = clf_tree, param_distributions = random_grid, n_iter = 100, cv = 5,scoring="precision")
clf_tree_random_f1= RandomizedSearchCV(estimator = clf_tree, param_distributions = random_grid, n_iter = 100, cv = 5,scoring="f1")

#Comenzamos por el DataFrame X
#Generamos conjuntos de train y de test. Para el test usamos el 20% de las observaciones
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

clf_tree_random_p.fit(X_train, y_train)

print("tuned hpyerparameters :(best parameters) ",clf_tree_random_p.best_params_)

clf_tree_p = clf_tree_random_p.best_estimator_  
clf_tree_p.fit(X_train,y_train)

eval_modelo_2 (clf_tree_p,X_train,y_train, X_test,y_test)

X.columns[clf_tree_p.feature_importances_>0.10]

#Optimizamos ahora el F1 Score

clf_tree_random_f1.fit(X_train, y_train)

print("tuned hpyerparameters :(best parameters) ",clf_tree_random_f1.best_params_)

clf_tree_f1 = clf_tree_random_f1.best_estimator_  
clf_tree_f1.fit(X_train,y_train)

eval_modelo_2 (clf_tree_f1,X_train,y_train, X_test,y_test)

X.columns[clf_tree_f1.feature_importances_>0.10]

#Parece que los modelos anteriores overfittean. Vamos a probar un modelo más sencillo
clf_tree = DecisionTreeClassifier(min_samples_leaf=20,max_depth=3)
clf_tree.fit(X_train,y_train)
eval_modelo_2(clf_tree,X_train,y_train, X_test,y_test)

X.columns[clf_tree_f1.feature_importances_>0.10]